Weight

In [9]:
import numpy as np

w = np.loadtxt("w0.fc1.weight.csv", delimiter=",")

w = w.reshape(784,128)

In [10]:
w.shape

(784, 128)

In [30]:
w[:8,:4]

array([[  1.,   2.,  -1.,   1.],
       [ -1.,  14.,   0.,   5.],
       [  5.,   0.,  12.,  12.],
       [-11.,   1.,   6.,  16.],
       [  0.,  -2.,   6.,   8.],
       [ -2.,  10.,   5., -10.],
       [ 11.,  -6.,  -6.,  -4.],
       [  5.,   8.,  -2.,   3.]])

In [29]:
w[784-8:, :4]

array([[  2.,  -4.,  -6.,  -9.],
       [  9., -12.,   6., -13.],
       [  3.,  -3.,   4.,  -1.],
       [ 11.,  -7.,   9.,  -1.],
       [ -7., -14., -29., -18.],
       [ 14.,  24.,  12.,  23.],
       [  3.,  16.,   4.,   0.],
       [ -3., -11.,  -6., -20.]])

In [32]:
w[:8, 4:8]

array([[  1.,  -4.,   0., -10.],
       [-10.,  -5.,   8.,   0.],
       [  3.,  21.,  -9., -17.],
       [ 10.,  -6.,   2.,  -8.],
       [ -5.,   5.,   1.,   5.],
       [-10.,   7.,  -8.,   9.],
       [  6.,  10.,  10.,   0.],
       [ -7.,   1.,  -8., -11.]])

In [33]:
w[784-8:, 4:8]

array([[  3.,  -6., -12.,   4.],
       [ 11.,  -3.,   1., -11.],
       [ -3.,  -1.,   4., -12.],
       [  9.,  11.,  19.,   1.],
       [-21.,  -7.,   0.,   2.],
       [  4., -15., -46., -88.],
       [  2.,  -4.,  -7.,  -1.],
       [-10., -33., -15., -17.]])

In [13]:
def make_instr(opvalid, opcode, sel, addr, data):
    return ((opvalid & 1) << 31) | ((opcode & 7) << 28) | \
           ((sel & 0xF) << 24) | ((addr & 0xFFFF) << 8) | (data & 0xFF)

In [14]:
OPCODE_LDSRAM = 0x2
NUM_BANK = 4
OC = 128
IC = 784

instr = []

for oc in range(OC):
    bank = oc % NUM_BANK          # 0~3 반복
    bank_row = oc // NUM_BANK     # OC=0,4,8... → row 증가

    for ic in range(IC):
        addr = bank_row * IC + ic   # 각 row는 784 길이
        d = int(w[ic][oc]) & 0xFF
        inst = make_instr(1, OPCODE_LDSRAM, bank, addr, d)
        instr.append(inst)

In [30]:
with open("instr_mem_temp.hex", "w") as f:
    for inst in instr:
        f.write(f"{inst:08x}\n")

MNIST

In [34]:
import numpy as np

# CSV: label + 784 pixels
data = np.loadtxt("mnist_test_small copy.csv", delimiter=",")
labels = data[:, 0].astype(int)
inputs = data[:, 1:]           # shape = (10, 784) for 10 images

In [35]:
inputs.shape

(10, 784)

In [36]:
inputs[0].shape

(784,)

In [50]:
def make_instr(opvalid, opcode, sel, addr, data):
    return ((opvalid & 1) << 31) | ((opcode & 7) << 28) | \
           ((sel & 0xF) << 24) | ((addr & 0xFFFF) << 8) | (data & 0xFF)

# 첫 번째 MNIST 이미지 사용 (784개)
img = inputs[0]             # shape = (784,)
# img = np.zeros_like(img)
# img[0] = 1
# img[-8] = 2
img_q = img.astype(np.int8) # 또는 네 quant 함수 적용

NUM_BANK = 8
input_instr = []

for idx in range(784):
    bank = idx % NUM_BANK
    addr = idx // NUM_BANK   # integer division
    data = int(img_q[idx]) & 0xFF  # 반드시 0~255
    inst = make_instr(1, OPCODE_LDSRAM, bank, addr, data)
    input_instr.append(inst)

In [51]:
with open("instr_input.hex", "w") as f:
    for inst in input_instr:
        f.write(f"{inst:08x}\n")

Result

In [52]:
import numpy as np

w = np.loadtxt("w0.fc1.weight.csv", delimiter=",")

# w = w[:16*8]
w.shape
# w = w.reshape(16,8)
w = w.reshape(784,128)

# CSV: label + 784 pixels
data = np.loadtxt("mnist_test_small copy.csv", delimiter=",")
labels = data[:, 0].astype(int)
inputs = data[:, 1:]           # shape = (10, 784) for 10 images

img = inputs[0]

print(w.shape)
print(img.shape)

(784, 128)
(784,)


In [58]:
img_row = img.reshape(1, 784)
out = img_row @ w    # (1,128)

In [59]:
out

array([[ 19057.,   3507.,  36036.,  23213.,  64202.,  59919.,  39223.,
         22298.,  59952.,  -2604.,   7842., -25514., -18027., -58322.,
        -25630., -11976., -20746., -21331.,   3897.,  -3527., -57378.,
        -63713., -46984.,   2895.,  18846.,  14963.,  17875.,   9987.,
         -7389.,  25462.,  59772.,  45856.,  70658.,  46311.,  38437.,
         32356.,  25024.,  -9190., -33755., -30049., -16804., -53059.,
        -44959., -32348., -57873.,  -9294., -18547., -23986., -26741.,
        -27177., -13935., -17631.,  12098., -18070., -14520.,  -3272.,
         28793.,  30182.,  75313.,  48746.,  40207.,  27128.,  21960.,
         32973.,  14798., -10493.,    632., -41714., -13926., -11179.,
        -30958., -25360., -18322.,  24038., -20559.,  15495., -33507.,
         -9198.,  -2901.,  -9194.,   8136.,  34433.,  -3025., -20535.,
         19572.,  36518.,  44161.,  18624.,  16962.,  16773.,  23692.,
        -11806.,   4935., -20035.,   -844., -13673.,  15398.,  -8574.,
      

In [56]:
import numpy as np

img_row = img.reshape(784).astype(np.int8)   # shape: (784,)
out = np.zeros(128)          # 결과 (128,)

for j in range(128):         # 출력 뉴런 128개
    s = 0
    for i in range(784):     # 입력 784개
        s += img_row[i] * w[i, j]
        if(img_row[i] != 0):
            print(f"i={i}, j={j}, img={img_row[i]}, w={w[i,j]}, partial_sum={s}")
    out[j] = s


i=0, j=0, img=1, w=1.0, partial_sum=1.0
i=1, j=0, img=2, w=-1.0, partial_sum=-1.0
i=2, j=0, img=3, w=5.0, partial_sum=14.0
i=202, j=0, img=84, w=-10.0, partial_sum=-826.0
i=203, j=0, img=-71, w=13.0, partial_sum=-1749.0
i=204, j=0, img=-97, w=-15.0, partial_sum=-294.0
i=205, j=0, img=-105, w=4.0, partial_sum=-714.0
i=206, j=0, img=60, w=-2.0, partial_sum=-834.0
i=207, j=0, img=36, w=1.0, partial_sum=-798.0
i=230, j=0, img=-34, w=29.0, partial_sum=-1784.0
i=231, j=0, img=-2, w=0.0, partial_sum=-1784.0
i=232, j=0, img=-2, w=7.0, partial_sum=-1798.0
i=233, j=0, img=-2, w=9.0, partial_sum=-1816.0
i=234, j=0, img=-2, w=-46.0, partial_sum=-1724.0
i=235, j=0, img=-15, w=-48.0, partial_sum=-1004.0
i=236, j=0, img=-58, w=17.0, partial_sum=-1990.0
i=237, j=0, img=-58, w=-32.0, partial_sum=-134.0
i=238, j=0, img=-58, w=2.0, partial_sum=-250.0
i=239, j=0, img=-58, w=9.0, partial_sum=-772.0
i=240, j=0, img=-58, w=-8.0, partial_sum=-308.0
i=241, j=0, img=-58, w=14.0, partial_sum=-1120.0
i=242, j=0, 

In [57]:
out

array([ -2191.,  -1101.,   7364.,   3245.,  -5430.,  -4337.,     55.,
         3610.,  -9936.,   2772.,   9122.,   6230.,   3733.,  -2258.,
          738.,   -968.,  -1290.,   2733.,   4921.,  -5319.,    734.,
        -3297.,  -1672.,  -4017.,   3742.,  -9869.,    -45.,    771.,
         3875.,   4982.,    636.,  -2272.,   -510.,   -537.,   4645.,
        -1436.,  -6976.,   4634.,   5157.,  -2657.,   1116.,   1981.,
         9569.,  -2908.,   3055.,   -846.,  -2163.,   8270.,  -2421.,
         2007.,  -5743.,   2337.,   2114.,   1386.,  -4280.,   6200.,
         7801.,   1510.,  -5583.,  -2454.,  -3057.,   2040.,  -5944.,
       -16435.,   -306.,  -6141.,  -7560.,   3598.,   9370.,   2133.,
        -3054.,   2288.,   1646.,   -282.,   4785.,   2439.,   8989.,
        -2030.,   7339.,  -1514.,   2760.,  -2943.,   7983.,   5065.,
        11380.,   4262.,   -639.,  -2880.,   7746.,    645.,  -3188.,
         1506.,  -5049.,  -8259.,  -5196.,   1943.,   5414.,  -3198.,
        -2309.,  -11

In [6]:
w

array([[  1.,   2.,  -1., ..., -14.,   1.,  -5.],
       [ -1.,  14.,   0., ...,  10., -10.,   2.],
       [  5.,   0.,  12., ...,   5., -19., -15.],
       ...,
       [ 14.,  24.,  12., ...,  19.,  10.,   6.],
       [  3.,  16.,   4., ...,   1.,  19.,  -1.],
       [ -3., -11.,  -6., ...,  -1.,  -5.,   2.]], shape=(784, 128))

In [7]:
print(out)
print(out.shape)

[[ 19057.   3507.  36036.  23213.  64202.  59919.  39223.  22298.  59952.
   -2604.   7842. -25514. -18027. -58322. -25630. -11976. -20746. -21331.
    3897.  -3527. -57378. -63713. -46984.   2895.  18846.  14963.  17875.
    9987.  -7389.  25462.  59772.  45856.  70658.  46311.  38437.  32356.
   25024.  -9190. -33755. -30049. -16804. -53059. -44959. -32348. -57873.
   -9294. -18547. -23986. -26741. -27177. -13935. -17631.  12098. -18070.
  -14520.  -3272.  28793.  30182.  75313.  48746.  40207.  27128.  21960.
   32973.  14798. -10493.    632. -41714. -13926. -11179. -30958. -25360.
  -18322.  24038. -20559.  15495. -33507.  -9198.  -2901.  -9194.   8136.
   34433.  -3025. -20535.  19572.  36518.  44161.  18624.  16962.  16773.
   23692. -11806.   4935. -20035.   -844. -13673.  15398.  -8574.  -8709.
  -15469. -18615.  -5486. -18660.  11969.    110.  17249.  32342.  46245.
   49111.  -8030.   5343.  22428.  12624. -36752.  25329.  17345. -11574.
  -13369.  -9074.   4040. -16035.  -93

1x8 하고 8x4만 때어서 계산

In [3]:
# ------------------------------
# 1) 입력 앞부분 8개 추출 → shape (8,)
# ------------------------------
img_part = img[:8]

# ------------------------------
# 2) weight 앞부분 8×4 추출 → shape (8,4)
# ------------------------------
w_part = w[:8, :4]

# ------------------------------
# 3) 행렬곱 (1×8) · (8×4) → (1×4)
# ------------------------------
out = img_part @ w_part   # 또는 np.dot(img_part, w_part)

print("img_part shape =", img_part.shape)   # (8,)
print("w_part shape =", w_part.shape)       # (8, 4)
print("out shape =", out.shape)             # (4,)
print(out)

img_part shape = (8,)
w_part shape = (8, 4)
out shape = (4,)
[0. 0. 0. 0.]


In [7]:
# ------------------------------
# 1) 전체 입력 1×784 → shape (784,)
# ------------------------------
img_full = img   # 이미 img = inputs[0] 이므로 (784,)

# ------------------------------
# 2) weight 앞부분 784×4 추출 → shape (784,4)
# ------------------------------
w_part = w[:, :4]

# ------------------------------
# 3) 행렬곱 (1×784) · (784×4) → (1×4)
# ------------------------------
out = img_full @ w_part   # 또는 np.dot(img_full, w_part)

print("img shape =", img_full.shape)   # (784,)
print("w_part shape =", w_part.shape)  # (784, 4)
print("out shape =", out.shape)        # (4,)
print(out)

img shape = (784,)
w_part shape = (784, 4)
out shape = (4,)
[19043.  3477. 36001. 23166.]
